In [1]:
# default_exp cli


# CLI tools

As explained in [nbdev docs](https://nbdev.fast.ai/tutorial.html#Set-up-console-scripts) and in [the @call_parse docs](https://fastcore.fast.ai/script.html) we can have `nbdev` set up command line tools for us.

In [2]:
#export
from fastcore.script import call_parse

In [3]:
# export
@call_parse
def rename(file):
    """Rename notebooks to base 1 for chapters."""
    print("Renaming notebooks now.")


In [4]:
# export
@call_parse
def add_copyright():
    """Add copyright string to all specified files."""
    print("Addding copyright string.")
